# Transformadores para sistema de recomendaciones

In [1]:
import numpy as np 
import pandas as pd 
#import kagglehub
#import os
import transformers
import torch



## Descargar conjunto de datos y lectura con pandas

In [2]:
'''
estas líneas están comentadas porque otra de las notebooks ya tiene código para descargar y guardar de forma local en otra carpeta los conjuntos de datos.
Si no se encuentran las carpetas, se pueden descomentar estas tres celdas y comentar las últimas dos líneas de la tercera

path = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")

print("Path to dataset files:", path)
'''

'\nestas líneas están comentadas porque otra de las notebooks ya tiene código para descargar y guardar de forma local en otra carpeta los conjuntos de datos.\nSi no se encuentran las carpetas, se pueden descomentar estas tres celdas y comentar las últimas dos líneas de la tercera\n\npath = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")\n\nprint("Path to dataset files:", path)\n'

In [3]:
'''extracted_directory = "/home/m/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1"
print(os.listdir(extracted_directory))
'''

'extracted_directory = "/home/m/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1"\nprint(os.listdir(extracted_directory))\n'

In [2]:
'''file_path_ratings = os.path.join(extracted_directory,'Books_rating.csv')
file_path_data = os.path.join(extracted_directory, 'books_data.csv')

ratings = pd.read_csv(file_path_ratings)
metadata = pd.read_csv(file_path_data)'''

metadata = pd.read_csv("/home/matias/Documentos/Mineria_de_textos/Proyecto_final/data/amazon-books-reviews/books_data.csv")
ratings = pd.read_csv("/home/matias/Documentos/Mineria_de_textos/Proyecto_final/data/amazon-books-reviews/Books_rating.csv")


## Exploración y limpieza de datos

In [3]:
metadata.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [5]:
metadata.shape

(212404, 10)

In [4]:
ratings.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [7]:
ratings.isnull().sum()

Id                          0
Title                     208
Price                 2518829
User_id                561787
profileName            561905
review/helpfulness          0
review/score                0
review/time                 0
review/summary            407
review/text                 8
dtype: int64

In [8]:
metadata.isnull().sum()

Title                 1
description       68442
authors           31413
image             52075
previewLink       23836
publisher         75886
publishedDate     25305
infoLink          23836
categories        41199
ratingsCount     162652
dtype: int64

In [9]:
ratings.shape

(3000000, 10)

In [10]:
#Quitamos valores faltantes para id de usuarios

ratings = ratings.dropna(subset=['User_id'])

ratings.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [11]:
ratings.isnull().sum()

Id                          0
Title                     195
Price                 2023633
User_id                     0
profileName               118
review/helpfulness          0
review/score                0
review/time                 0
review/summary            397
review/text                 1
dtype: int64

In [12]:
#Revisemos entradas repetidas y eliminemos las repeticiones

duplicados = ratings.duplicated()

print("Número de duplicados: ", duplicados.sum())

Número de duplicados:  5546


In [13]:
ratings = ratings[~duplicados]

ratings.shape

(2432667, 10)

In [37]:
usuarios = ratings['User_id'].nunique()


AttributeError: 'int' object has no attribute 'shape'

In [38]:
usuarios

1008972

In [14]:
# Cambiar el formato de 'review/time' a datetime
ratings['review/time'] = pd.to_datetime(ratings['review/time'], unit='s')

# Ordenar por 'review/time' de manera ascendente
ratings = ratings.sort_values(by='review/time', ascending=True)

ratings

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
2971448,B000G167FA,Silver Pennies,NaN,AAFZZHA2I598B,Byron C. Benson,4/4,5.0,1969-12-31 23:59:59,An incomparable children's classic,This book of children's poems has been enjoyed...
2971449,B000G167FA,Silver Pennies,NaN,A1MCQGDJDSPJFF,SHOCKgBLUE@aol.com,4/4,5.0,1969-12-31 23:59:59,Unequalled Collection of Children's Poetry for...,I was introduced to this marvelous collection ...
75754,0786280670,"Julie and Julia: 365 Days, 524 Recipes, 1 Tiny...",NaN,A2FBKQ7J7D0Z36,Angie789,7/10,2.0,1969-12-31 23:59:59,disappointing,"I love both cooking and reading blogs, so I th..."
75753,0786280670,"Julie and Julia: 365 Days, 524 Recipes, 1 Tiny...",NaN,A7PNOKN67P7T3,"Mary Harris ""beachreader""",7/10,1.0,1969-12-31 23:59:59,waste of time and money,A book as messy and nasty as some of the autho...
2152018,B000KPX7RI,"McKeachie's Teaching Tips, Strategies Research...",NaN,A1AY4QM3FDINBQ,Sujini2,0/0,5.0,1969-12-31 23:59:59,Pretty good.,Pretty good at providing me with the fundament...
...,...,...,...,...,...,...,...,...,...,...
1017673,155927140X,The Enchanted Cat,NaN,AOS6V51CJM8NQ,"Ted M. Sweitzer ""Judy Sweitzer""",0/0,4.0,2013-03-04 00:00:00,For the Cat Lover,"A nice gift book, my husband bought it for me ..."
1564116,1423315537,Firehouse,NaN,AO17E7H0S0PBU,TiffanyW,0/0,5.0,2013-03-04 00:00:00,Love this book,"This book is a poignant, emotional, humorous, ..."
960831,1850891648,Frankenstein (Isis Large Print Fiction),NaN,A3SFQL1UXFOQY,Kylee Bennett,0/0,5.0,2013-03-04 00:00:00,Good,"This is my favorite, it is very eventful and i..."
1888684,0435130994,Skeleton Key (New Windmills),NaN,A1F8YGI6MX27KC,Sharon Rogers,0/0,5.0,2013-03-04 00:00:00,Amazing,This was action packed from beginning to end.T...


In [15]:
#Agrupamos el data frame por el id del usuario

ratingsAgrupados = ratings.groupby(['User_id'], sort=False)
ratingsAgrupados.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
2971448,B000G167FA,Silver Pennies,NaN,AAFZZHA2I598B,Byron C. Benson,4/4,5.0,1969-12-31 23:59:59,An incomparable children's classic,This book of children's poems has been enjoyed...
2971449,B000G167FA,Silver Pennies,NaN,A1MCQGDJDSPJFF,SHOCKgBLUE@aol.com,4/4,5.0,1969-12-31 23:59:59,Unequalled Collection of Children's Poetry for...,I was introduced to this marvelous collection ...
75754,0786280670,"Julie and Julia: 365 Days, 524 Recipes, 1 Tiny...",NaN,A2FBKQ7J7D0Z36,Angie789,7/10,2.0,1969-12-31 23:59:59,disappointing,"I love both cooking and reading blogs, so I th..."
75753,0786280670,"Julie and Julia: 365 Days, 524 Recipes, 1 Tiny...",NaN,A7PNOKN67P7T3,"Mary Harris ""beachreader""",7/10,1.0,1969-12-31 23:59:59,waste of time and money,A book as messy and nasty as some of the autho...
2152018,B000KPX7RI,"McKeachie's Teaching Tips, Strategies Research...",NaN,A1AY4QM3FDINBQ,Sujini2,0/0,5.0,1969-12-31 23:59:59,Pretty good.,Pretty good at providing me with the fundament...
...,...,...,...,...,...,...,...,...,...,...
1017673,155927140X,The Enchanted Cat,NaN,AOS6V51CJM8NQ,"Ted M. Sweitzer ""Judy Sweitzer""",0/0,4.0,2013-03-04 00:00:00,For the Cat Lover,"A nice gift book, my husband bought it for me ..."
1564116,1423315537,Firehouse,NaN,AO17E7H0S0PBU,TiffanyW,0/0,5.0,2013-03-04 00:00:00,Love this book,"This book is a poignant, emotional, humorous, ..."
960831,1850891648,Frankenstein (Isis Large Print Fiction),NaN,A3SFQL1UXFOQY,Kylee Bennett,0/0,5.0,2013-03-04 00:00:00,Good,"This is my favorite, it is very eventful and i..."
1888684,0435130994,Skeleton Key (New Windmills),NaN,A1F8YGI6MX27KC,Sharon Rogers,0/0,5.0,2013-03-04 00:00:00,Amazing,This was action packed from beginning to end.T...


In [16]:
ratingsAgrupados.get_group('AVCGYZL8FQQTD')

/tmp/ipykernel_3623/3120651494.py:1: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  ratingsAgrupados.get_group('AVCGYZL8FQQTD')


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
1282430,1882931335,Six Foot One and Worth the Climb,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",16/18,4.0,1999-10-17,"If you like Julie, you'll like this pictorial ...",A pictorial exploration of Julie. There is a l...
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,1999-10-23,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1293256,B000TDGZO8,Under Fire - A Corps Novel,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",2/3,4.0,2005-02-03,tired characters - good stories,Basically I want to reiterate what most everyo...


In [32]:
ratingsAgrupados.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
2971448,B000G167FA,Silver Pennies,NaN,AAFZZHA2I598B,Byron C. Benson,4/4,5.0,1969-12-31 23:59:59,An incomparable children's classic,This book of children's poems has been enjoyed...
2971449,B000G167FA,Silver Pennies,NaN,A1MCQGDJDSPJFF,SHOCKgBLUE@aol.com,4/4,5.0,1969-12-31 23:59:59,Unequalled Collection of Children's Poetry for...,I was introduced to this marvelous collection ...
75754,0786280670,"Julie and Julia: 365 Days, 524 Recipes, 1 Tiny...",NaN,A2FBKQ7J7D0Z36,Angie789,7/10,2.0,1969-12-31 23:59:59,disappointing,"I love both cooking and reading blogs, so I th..."
75753,0786280670,"Julie and Julia: 365 Days, 524 Recipes, 1 Tiny...",NaN,A7PNOKN67P7T3,"Mary Harris ""beachreader""",7/10,1.0,1969-12-31 23:59:59,waste of time and money,A book as messy and nasty as some of the autho...
2152018,B000KPX7RI,"McKeachie's Teaching Tips, Strategies Research...",NaN,A1AY4QM3FDINBQ,Sujini2,0/0,5.0,1969-12-31 23:59:59,Pretty good.,Pretty good at providing me with the fundament...
...,...,...,...,...,...,...,...,...,...,...
1017673,155927140X,The Enchanted Cat,NaN,AOS6V51CJM8NQ,"Ted M. Sweitzer ""Judy Sweitzer""",0/0,4.0,2013-03-04 00:00:00,For the Cat Lover,"A nice gift book, my husband bought it for me ..."
1564116,1423315537,Firehouse,NaN,AO17E7H0S0PBU,TiffanyW,0/0,5.0,2013-03-04 00:00:00,Love this book,"This book is a poignant, emotional, humorous, ..."
960831,1850891648,Frankenstein (Isis Large Print Fiction),NaN,A3SFQL1UXFOQY,Kylee Bennett,0/0,5.0,2013-03-04 00:00:00,Good,"This is my favorite, it is very eventful and i..."
1888684,0435130994,Skeleton Key (New Windmills),NaN,A1F8YGI6MX27KC,Sharon Rogers,0/0,5.0,2013-03-04 00:00:00,Amazing,This was action packed from beginning to end.T...


In [34]:
# Armar los historiales de reseñas por usuario
user_histories = {}
for user_id, group in ratingsAgrupados:
    user_histories[user_id[0]] = list(zip(group['Title'], group['review/score'], group['review/time']))

In [35]:
"AVCGYZL8FQQTD" in user_histories

True

In [36]:
user_histories["AVCGYZL8FQQTD"]

[('Six Foot One and Worth the Climb', 4.0, Timestamp('1999-10-17 00:00:00')),
 ('Its Only Art If Its Well Hung!', 4.0, Timestamp('1999-10-23 00:00:00')),
 ('Under Fire - A Corps Novel', 4.0, Timestamp('2005-02-03 00:00:00'))]

In [26]:
#Quitar usuarios con 1 review

usuarios =user_histories


User_id
A14OJS0VWMOSWO    5747
AFVQZQ8PW0L       3602
A1D2C0WDCSHUWZ    3139
AHD101501WCN1     1995
A1X8VZWTOG8IS6    1804
                  ... 
AOS6V51CJM8NQ        1
AO17E7H0S0PBU        1
A3SFQL1UXFOQY        1
A1F8YGI6MX27KC       1
A38BU28ZG0Z9S4       1
Name: count, Length: 1008972, dtype: int64